In [21]:
# ---
# PREPARE DATA:
promptToContinue <- function(msg){
  msg.suffix <- ": To stop, enter no: "
  toContinue <- tolower(readline(paste(msg, msg.suffix)))
  stopifnot(toContinue != "no")
  # else quietly continue
  return()
}
getRowRightTot <- function(k, qaTable){
  qaTable.k <- qaTable[k,]
  right.k <- qaTable.k$correct # init
  tot.k <- qaTable.k$wrong + qaTable.k$unattempted + qaTable.k$correct # init
  if(!(is.na(qaTable.k$partCorrect1m) || is.na(qaTable.k$partCorrect2m) || is.na(qaTable.k$partCorrect3m))){
    # has partCorrect* data. beware: is.na() doesn't consider vectors etc
    # stopifnot(qaTable.k$markScheme == "-201234")
    # "-103" etc don't have partCorrect* data! and there seems to be just two questions (173, 245) with partCorrect3m!
    # if 3-marker questions have partCorrect* data, following formula has to be amended
    right.k <- right.k + round(1/4 * qaTable.k$partCorrect1m + 2/4 * qaTable.k$partCorrect2m
      + 3/4 * qaTable.k$partCorrect3m, 0) # fractional-marks-weighted sum
    tot.k <- tot.k + (qaTable.k$partCorrect1m + qaTable.k$partCorrect2m + qaTable.k$partCorrect3m) # not weighted sum
  } # else continue
  rightTot.k <- c(as.integer(right.k), as.integer(tot.k))
  return(rightTot.k)
}
getRightTot <- function(qaTable){
  rightTot <- mapply(getRowRightTot, c(1:nrow(qaTable)), MoreArgs=list(qaTable))
  return(t(rightTot))
}
clog <- function(x) log(x + 0.5) # from ref, in case of zeroes
cfac <- function(x, breaks = NULL) { # from ref, which by default tries to take an educated guess how to choose the
 # breaks between the categories
 if(is.null(breaks)) breaks <- unique(quantile(x, 0:10/10))
 x <- cut(x, breaks, include.lowest = TRUE, right = FALSE)
 levels(x) <- paste(breaks[-length(breaks)], ifelse(diff(breaks) > 1,
   c(paste("-", breaks[-c(1, length(breaks))] - 1, sep = ""), "+"), ""),
   sep = "")
 return(x)
}
cDecimalDigits <- 3 # by default

JEEdata <- read.csv( file="qaJEEadvanced.csv", stringsAsFactors=TRUE) # 684 obs. of  13 variables
str(JEEdata)

JEEdata.use <- JEEdata[,-1] # keep original as (read-in) is, as master reference, and take a copy for use
  # eg: within(cancer, {
  #   grade <- factor(grade, 0:2, c("well differentiated","moderately differentiated","poorly differentiated"),
  #     ordered = TRUE) # for ordinal data
  #   race <- factor(...) ...})
ms.chr <- JEEdata.use$markScheme
ms.fac <- as.factor(sub("^2", "+02", sub("^3", "+03", ms.chr))); summary(ms.fac)
JEEdata.use$markScheme <- ms.fac
JEEdata.use$year <- as.factor(JEEdata.use$year)
  # consider 2015 missing etc. Beware: not considered as ordered, which could result in different models vs unordered
JEEdata.use$paper <- as.factor(JEEdata$paper)
rightTot <- getRightTot(JEEdata.use); right <- rightTot[,1]; tot <- rightTot[,2]
rOtot <- round(right / tot, cDecimalDigits)
# wOtot <- round(JEEdata.use$wrong / tot, cDecimalDigits)
# uOtot <- round(JEEdata.use$unattempted / tot, cDecimalDigits)
# cOtot <- round(JEEdata.use$correct / tot, cDecimalDigits)
# rOtot <- round(right / tot, cDecimalDigits)
# nonwucOtot <- 1 - (wOtot + uOtot + cOtot)
# nonwurOtot <- 1 - (wOtot + uOtot + rOtot)
# JEEdata.use <- cbind(JEEdata.use, right=right, tot=tot, wOtot=wOtot, uOtot=uOtot, cOtot=cOtot, rOtot=rOtot)
# JEEdata.use <- cbind(JEEdata.use, nonwucOtot=nonwucOtot, nonwurOtot=nonwurOtot)
# wur <- JEEdata.use$wrong + JEEdata.use$unattempted + right
# rOwu <- round(right / with(JEEdata.use, wrong + unattempted), cDecimalDigits)
JEEdata.use <- cbind(JEEdata.use, # wu=(wur - right), # wu was named wurNotRight
  right=right, tot=tot,
  wOtot=round(JEEdata.use$wrong / tot, cDecimalDigits),
  uOtot=round(JEEdata.use$unattempted / tot, cDecimalDigits),
  cOtot=round(JEEdata.use$correct / tot, cDecimalDigits),
  rOtot=rOtot,
  not.rOtot=(1-rOtot) # (density of) those who got partially correct without getting anything wrong
)

JEEdata.use <- JEEdata.use[JEEdata.use$unattempted > 0,]; nrow(JEEdata.use)
  # JEEdata[JEEdata$unattempted==0,] shows 5+2 rows from 2017 and 2012.
  # 2017's appear to be marked correct as bonus; 2012's appear dropped
colNames.outCount <- c("wrong", "unattempted", "correct", "right", "tot")
colNames.outRatio <- c("wOtot", "uOtot", "cOtot", "rOtot", "not.rOtot")
  # was: c("wOwur", "uOwur", "rOwur")
stopifnot(! (is.null(JEEdata.use[,colNames.outCount]) | is.na(JEEdata.use[,colNames.outCount])))
str(JEEdata.use); head(JEEdata.use)

'data.frame':	684 obs. of  13 variables:
 $ qNum         : Factor w/ 114 levels "1","10","11",..: 1 12 23 34 45 56 58 59 60 2 ...
 $ unattempted  : int  38510 42122 38242 78136 52714 17632 22879 36133 37834 29613 ...
 $ wrong        : int  51117 64996 69095 50125 29914 25057 77084 104174 110251 91112 ...
 $ correct      : int  56556 25284 30124 17372 13477 48285 55195 14851 7073 34433 ...
 $ partCorrect1m: int  8975 22756 17697 9525 27122 24786 NA NA NA NA ...
 $ partCorrect2m: int  0 0 0 0 31931 39398 NA NA NA NA ...
 $ partCorrect3m: int  0 0 0 0 0 0 NA NA NA NA ...
 $ subject      : Factor w/ 3 levels "chemistry","maths",..: 3 3 3 3 3 3 3 3 3 3 ...
 $ paper        : int  1 1 1 1 1 1 1 1 1 1 ...
 $ year         : int  2018 2018 2018 2018 2018 2018 2018 2018 2018 2018 ...
 $ qaNum        : int  1 2 3 4 5 6 7 8 9 10 ...
 $ qaType       : Factor w/ 7 levels "Compr","MultCorrAns",..: 2 2 2 2 2 2 3 3 3 3 ...
 $ markScheme   : int  -201234 -201234 -201234 -201234 -201234 -201234 3 3 3 3 ..

-103    -104 -201234     +02     +03 
    288      30     126      30     210

[1] 677

ERROR: Error: !(is.null(JEEdata.use[, colNames.outCount]) | is.na(JEEdata.use[,  .... are not all TRUE


In [ ]:
# ---
# RELATE OUT:
cProbs <- c(0, 0.01, 0.05, 0.1, 0.25, 0.333); cProbs <- sort(c(cProbs, 0.5, 1 - cProbs))
require(corrplot)
summary(JEEdata.use)
# for count data exploratory stat, use clog(). picture is much clearer if the dependent variable is log-transformed (just
# as all count regression models discussed above also use a log link by default)
warning("consider log(rOtot) and other similar responses ahead")
for(i in c("out ratios")){ # c("out counts", "out ratios") or c("out ratios")
  if(i=="out counts"){
    cOutNames <- colNames.outCount; cDecimalDigits <- 0
  } else {
    cOutNames <- colNames.outRatio; cDecimalDigits <- 3
  }
  print(paste("count, mean, sd, and percentiles for outputs:", paste(cOutNames, collapse=", ")))
  for(outName in cOutNames){
    out <- JEEdata.use[,outName]
    out.stat <- c(length(out), round(mean(out), cDecimalDigits),
      round(sd(out), cDecimalDigits), round(quantile(out, probs=cProbs), cDecimalDigits))
    print(out.stat)
    hist(out, xlab=outName)
    promptToContinue("continue")
  }

  cor.meth <- "spearman"
    # Kendall's tau or Spearman's rho statistic is used to estimate a rank-based measure of association. These are more
    # robust and have been recommended if the data do not necessarily come from a bivariate normal distribution.
    # method=c("pearson", "spearman", "kendall").
  myCor <- cor(JEEdata.use[,cOutNames], method=cor.meth)
    # consider right+unattempted as OUT
  print(round(myCor, 2)) # coz range [-1,1]
  corrplot(myCor, type="upper", order="hclust", # for just the upper triangular of correlation matrix
    title=paste("Correlation Plot using method", cor.meth, "with", paste(cOutNames, collapse=",")))
    # alt: require("PerformanceAnalytics"); chart.Correlation(my_data, histogram=TRUE, pch=19)
  promptToContinue("continue")
}
outRatio.fac <- with(JEEdata.use,
  cbind(as.data.frame(rOtot), wOtot.fac=as.factor(cfac(wOtot)), uOtot.fac=as.factor(cfac(uOtot)),
    cOtot.fac=as.factor(cfac(cOtot)), rOtot.fac=as.factor(cfac(rOtot)), not.rOtot.fac=as.factor(cfac(not.rOtot))))
plot(rOtot ~ wOtot.fac + uOtot.fac + cOtot.fac + rOtot.fac + not.rOtot.fac, data=outRatio.fac)
# outCount.fac <- with(JEEdata.use,
#   cbind(as.data.frame(right), w.fac=as.factor(cfac(wrong)), u.fac=as.factor(cfac(unattempted)),
#     c.fac=as.factor(cfac(correct)), r.fac=as.factor(cfac(right))))
# plot(right ~ w.fac + u.fac + c.fac, data=outCount.fac)


In [ ]:
# ---
# RELATE IN:
with(JEEdata.use, table(qaType, year)) # table(exclude=) levels to remove for all factors in ...
with(JEEdata.use, table(qaType, subject))
qaType.grp <- JEEdata.use$qaType
levels(qaType.grp) <- c("notMultSingCorrDigit", "MultCorrAns", "notMultSingCorrDigit", "notMultSingCorrDigit",
  "SingCorrAns", "SingDigitInteger", "notMultSingCorrDigit")
  # c("Compr", "MultCorrAns", "NumAns", "ParaSingleAns", "SingCorrAns", "SingDigitInteger", "TwoListMatchSingleAns")
  # considering year-wise samples: {"MultCorrAns", "SingCorrAns", "SingDigitInteger", others}
  # qaType is really unordered. so, ordering might be unsuitable
  # JEEdata.use$qaType <- ordered(JEEdata.use$qaType, levels=c("NumAns", "MultCorrAns", "SingDigitInteger",
  #   "ParaSingleAns", "Compr", "SingCorrAns", "TwoListMatchSingleAns"))
  # <- relevel(ml$prog, ref = "academic") # choose the level of our outcome that we wish to use as our baseline
with(JEEdata.use, table(markScheme, year)) # -ve and +ve markScheme could be grouped, but no finer coz biased sample
markScheme.grp <- JEEdata.use$markScheme
levels(markScheme.grp) <- c("-ve", "-ve", "-ve", "+ve", "+ve") # {-103 -104 -201234} {+02 +03} and no finer
JEEdata.use <- cbind(JEEdata.use, qaType.grp=qaType.grp, markScheme.grp=markScheme.grp)
# qat.sub.rOwu <- round(exp(xtabs(log(rOwu) ~ qaType + subject, data=JEEdata.use) /
#   with(JEEdata.use, table(qaType, subject))), cDecimalDigits)
# !is.list(replications(formula, JEEdata)) # eg, replications(~ . - yield, npk).  test for balance
# most non-experimentally-designed data is unbalanced
# lme()


In [ ]:
# ---
# MODEL OUT-IN:
cInNames <- c("subject", "paper", "year", "qaType", "qaType.grp", "markScheme.grp")
summary(JEEdata.use[,cInNames])
# cInNames <- cInNames[-(2:2)]
cOutNames <- "right" # "log(rOwu)"
mdl.spec <- as.formula(paste(cOutNames, "~", paste(cInNames, collapse=" + "))); mdl.spec
mdl.spec.plot <- update(mdl.spec, log(.) ~ .); mdl.spec.plot # replace LHS with log(LHS)
plot(mdl.spec.plot, data=JEEdata.use)
# plot(log(rOwu) ~ qaType * markScheme.grp, data=JEEdata.use)

  # ref: https://cran.r-project.org/web/packages/pscl/vignettes/countreg.pdf
  require(MASS); require(lmtest); require(sandwich); require(pscl)
  myData <- JEEdata.use; mdl.spec.bak <- mdl.spec
  contrasts(myData$qaType) <- contr.treatment(levels(myData$qaType), base=3) # "NumAns" coz that goes with lowest mean
  contrasts(myData$subject) <- contr.treatment(levels(myData$subject), base=2) # "maths" coz that goes with lowest mean
  # contr.treatment(n, base=1, contrasts=TRUE, sparse=FALSE); C()
  # ?glm says:
  # Non-NULL weights can be used to indicate that different observations have different dispersions (with the values in
  # weights being inversely proportional to the dispersions); or equivalently, when the elements of weights are positive
  # integers w_i, that each response y_i is the mean of w_i unit-weight observations. For a binomial GLM prior weights are
  # used to give the number of trials when the response is the proportion of successes: they would rarely be used for a
  # Poisson GLM.
  mdl.spec <- update(mdl.spec, ~ . + offset(log(tot)))
    # append offset(log(tot)) to indicate term with a fixed coefficient of one

  fit <- glm(mdl.spec, data=myData, family=poisson)
    # prefer rate of incidence (not count)
    # =loglm() cf Negative Binomial, possibly with Hurdle and Zero-Inflated Regression Models. package pscl?
    # family=negative.binomial(link="log")
    # Count data often have an exposure variable, which indicates the number of times the event could have happened.
    # This variable should be incorporated into a Poisson model with the use of the offset option.
  coeftest(fit, vcov=sandwich); summary(fit); plot(fit)

  fit <- glm(update(mdl.spec, ~ . - qaType.grp), data=myData, family=poisson)
    # drop qaType.grp coz NAs due to related qaType.grp: [Coefficients: (3 not defined because of singularities)]
  coeftest(fit, vcov=sandwich); summary(fit); plot(fit)

  fit <- MASS::glm.nb(mdl.spec, data=myData); m1 <- fit
  coeftest(fit, vcov=sandwich); summary(fit); plot(fit)
    # 2018::physics 1.7x r count as per NB
    # myData[c(44, 49, 66, ?99 if -subject?),] are plotted as outliers in
    # Residuals vs Fitted, Q-Q, & Scale-Location plots; investigate
    # myData[c(61, 261, 594),] are plotted as outliers in Residuals vs Leverage plot; investigate
    # prefer rate of incidence (not count). was: offset(log(wur))
    # ref https://mac-theobio.github.io/QMEE/Generalized_linear_models.html says of overdispersion:
    # "too much variance: (residual deviance)/(residual df) should be ˜1. (Ratio >1.2 worrisome; ratio>3,
    # v. worrisome (check your model & data!)"
    # so, 715.6/661=1.08 is therefore "not worrisome"; Dispersion parameter 2.89
    # ref https://mac-theobio.github.io/QMEE/Generalized_linear_models.html says of offsets:
    # "constant terms added to a model
    # what if we want to model densities rather than counts?
    # log-link (Poisson/NB) models: mu0=exp(beta0+beta1x1+...)
    # if we know the area then we want mu=A * mu0
    # equivalent to adding log(A) to the linear predictor (exp(log(mu0)+log(A))=mu0 * A)
    # use ... + offset(log(A)) in R formula"
    # so, by including offset(log(tot)), density of 'right' (over an "area" of 'tot' JEE candidates) is being modeled

  fit <- MASS::glm.nb(update(mdl.spec, ~ . - qaType.grp), data=myData)
    # drop qaType.grp coz NAs due to related qaType.grp: [Coefficients: (3 not defined because of singularities)]
  coeftest(fit, vcov=sandwich); summary(fit); plot(fit)

  fit <- MASS::glm.nb(update(mdl.spec, ~ . - qaType.grp - 1), data=myData)
    # drop qaType.grp coz NAs due to related qaType.grp: [Coefficients: (3 not defined because of singularities)]
    # drop intercept coz we want to estimate relative impacts of variables, not necessarily estimate the response itself
    #   (~ + 0) or (~ . - 1) can equivalently drop the intercept
  coeftest(fit, vcov=sandwich); summary(fit); plot(fit)
    # Null Deviance went up from 880 to 3394 after dropping intercept from model spec, while AIC stays 15434

  fit <- MASS::glm.nb(update(mdl.spec, ~ . - qaType.grp - 1 - paper), data=myData); m2 <- fit
    # drop paper too coz insignificant impact
  coeftest(fit, vcov=sandwich); summary(fit); plot(fit)

  est.betahat <- round(exp(cbind(Estimate=coef(fit), confint(fit))), cDecimalDigits); est.betahat
  # ref: https://stats.idre.ucla.edu/r/dae/negative-binomial-regression/
  # We might be interested in looking at incident rate ratios rather than coefficients. So, exponentiate
  # incident rate (IRR) for levelA is Est times the incident rate for the reference group
  # For a constant value of other variables, for every one-point increase in chosen variable (in the following),
  # estimated odds of modeled response are multiplied by ...  exp(betahat) iff logit link for Odds Ratio
  # betahat <- fit$coefficients; round(exp(betahat), cDecimalDigits)
  # The two degree-of-freedom chi-square test if < alpha=0.05 indicates that factor is a statistically significant
  # predictor of response
  fit <- MASS::glm.nb(update(mdl.spec, ~ . - qaType.grp - 1 - paper - subject), data=myData); m3 <- fit
  anova(m1, m2, m3) # factor subject is stat-significant predictor of response; so, avoid dropping subject, and prefer m2.
    # dropping paper and qaType.grp is ok coz they are not significant predictors
  fit <- m2 # preferred model

  # TBD: additionally, you can predict (and plot) for a data set to get what the model generates:
  # lines(myYear, predict(fit, type="response", list(year=myYear)))
  fit.pred.link <- predict(fit, type="link"); fit.pred.resp <- predict(fit, type="response")
  str(fit.pred.link); str(fit.pred.resp)

  # fit <- glm(rOtot ~ ., weights=tot, family=binomial(link="logit"))
    # As a numerical vector with values between 0 and 1, interpreted as the proportion of successful cases (with the
    # total number of cases given by the weights)
  # fit <- glm(cbind(right, (tot - right)) ~ ., data=myData, weights=tot, family=binomial(link="logit"))
    # As a two-column integer matrix: the first column gives the number of successes and the second the number of failures
    # weights when response has counts, e.g., for binomial GLM, prior weights are used to give number of trials

In [ ]:
# investigate outliers from diagnostic plots and theorize on extreme observations, e.g., 44, 66, etc
# ref http://www.contrib.andrew.cmu.edu/~achoulde/94842/homework/regression_diagnostics.html.
# [[Those "Cook's distance" dashed curves don't even appear on the plot.]]
# else the points that lie close to or outside of the dashed red curves are worth investigating further.
myData.o1 <- myData[c(44, 49, 66),] # are plotted as outliers in Residuals vs Fitted, Q-Q, & Scale-Location plots
myData.o2 <- myData[c(61, 261, 594),] # are plotted as outliers in Residuals vs Leverage plot
myData.o1; myData.o2
